In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('landmarks.csv')

# Separate features and labels
X = df.drop('label', axis=1).values  # Features (landmarks)
y = df['label'].values  # Labels (class 0-9, A-Z)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


Training data shape: (1296, 63)
Testing data shape: (325, 63)


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_encoded = to_categorical(y_train_encoded, num_classes=36)
y_test_encoded = to_categorical(y_test_encoded, num_classes=36)

# Reshape input data for CNN-LSTM
# Reshaping input data: (samples, timesteps, landmarks, features, channels)
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, 21, 3, 1))  # 1 timestep, 21 landmarks, 3 features
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, 21, 3, 1))

# Build the CNN-LSTM model
model = models.Sequential()

# TimeDistributed CNN layers for feature extraction (applied to each time step)
model.add(layers.TimeDistributed(layers.Conv2D(32, (2, 2), activation='relu', padding='same'), input_shape=(1, 21, 3, 1)))
model.add(layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2, 1))))  # Smaller pooling to preserve dimensions
model.add(layers.TimeDistributed(layers.Conv2D(64, (2, 2), activation='relu', padding='same')))
model.add(layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2, 1))))  # Adjust pool size to avoid over-reduction
model.add(layers.TimeDistributed(layers.Flatten()))

# LSTM layer for temporal sequence processing
model.add(layers.LSTM(128, return_sequences=False))  # LSTM layer with 128 units

# Fully connected layers
model.add(layers.Dense(64, activation='relu'))  # Dense layer with 64 neurons
model.add(layers.Dropout(0.3))  # Dropout layer for regularization
model.add(layers.Dense(32, activation='relu'))  # Dense layer with 32 neurons
model.add(layers.Dropout(0.3))

# Output layer for classification (36 classes: 0-9 and A-Z)
model.add(layers.Dense(36, activation='softmax'))  # Output layer for 36 classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Since this is a multi-class classification problem
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train_reshaped, y_train_encoded, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_test_encoded))

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded)
print(f"Test Accuracy: {test_accuracy:.4f}")


d:\PythonProject\signlanguage\.venv\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_3              │ (None, 1, 21, 3, 32)   │           160 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 1, 10, 3, 32)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 1, 10, 3, 64)   │         8,256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 1, 5, 3, 64)    │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 1, 960)         │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       557,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 36)             │         1,188 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 577,508 (2.20 MB)

 Trainable params: 577,508 (2.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.0295 - loss: 3.5733 - val_accuracy: 0.0431 - val_loss: 3.4970
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0443 - loss: 3.4793 - val_accuracy: 0.1077 - val_loss: 3.3230
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0771 - loss: 3.3242 - val_accuracy: 0.1908 - val_loss: 2.9693
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0968 - loss: 3.0402 - val_accuracy: 0.2708 - val_loss: 2.4486
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.1907 - loss: 2.6449 - val_accuracy: 0.3815 - val_loss: 2.0296
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.2806 - loss: 2.2911 - val_accuracy: 0.5446 - val_loss: 1.6410
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3511 - loss: 1.9512 - val_accuracy: 0.6062 - val_loss: 1.4341
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4095 - loss: 1.6753 - val_accuracy: 0.

In [7]:
model.save("sign_model2.h5")